In [ ]:
import os
from libv3.utils import *
import pandas as pd
import json

In [ ]:
#### paths to all files
normal_vpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/normal/varlist_trial3.json'

faulty1_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial1'
faulty2_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial2'
faulty3_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial3'
faulty4_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial4'

faulty1_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial1.json'
faulty2_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial2.json'
faulty3_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial3.json'
faulty4_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial4.json'

normal1_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/normal/trace_trial1'
normal2_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/normal/trace_trial2'
normal3_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/normal/trace_trial3'

normal1_vpath = '../trace_data/mamba2/single_thread/version_3/normal/varlist_trial1.json'
normal2_vpath = '../trace_data/mamba2/single_thread/version_3/normal/varlist_trial2.json'
normal3_vpath = '../trace_data/mamba2/single_thread/version_3/normal/varlist_trial3.json'


In [ ]:
#### load varlist for normal trace
normal_to_number = read_json(normal_vpath)

if normal_to_number != False:
    normal_from_number = mapint2var(normal_to_number)

In [ ]:
len(normal_from_number)

In [ ]:
#### load faulty1 trace and varlist
paths_to_varlist = [
                    # faulty1_vpath,
                    faulty2_vpath,
                    faulty3_vpath, 
                    faulty4_vpath,
                    normal1_vpath,
                    normal2_vpath,
                    normal3_vpath
                    ]
paths_to_trace = [
                    # faulty1_tpath, 
                  faulty2_tpath, 
                  faulty3_tpath, 
                  faulty4_tpath,
                    normal1_tpath,
                    normal2_tpath,
                    normal3_tpath
                  ]



for path_to_varlist, path_to_trace in zip(paths_to_varlist, paths_to_trace):
    ### check if pair of trace and varlist correct
    assert( os.path.basename(path_to_trace).split('_')[1] in path_to_varlist )
    #### check if varlist consistent with normal trace
    check_consistence, _ = is_consistent([normal_vpath, path_to_varlist])

    if check_consistence != False:
        print(f"{os.path.basename(path_to_trace).split('_')[1]} varlist is same as normal")
    else:
        print(f"correcting the trace {os.path.basename(path_to_trace).split('_')[1]} to work with normal varlist:", path_to_trace)
        faulty_trace = read_traces(path_to_trace)

        faulty_to_number = read_json(path_to_varlist)
        faulty_from_number = mapint2var(faulty_to_number)

        ### get trace with varibale names
        faulty_vartrace = []
        for stateint, ts in faulty_trace:
            # print(stateint, ts)
            state_var = faulty_from_number[stateint]
            faulty_vartrace += [[state_var, ts]]

        ### check if the length of the trace is same. We compare trace with the variable names and trace with the numbers
        assert (len(faulty_vartrace) == len(faulty_trace))

        ### use the varlist same as the normal trace for consistency. If the variable is not in the normal varlist, add it to the normal varlist
        faulty_trace_new = []
        state_var = ''
        state_int = 0
        max_int = max(normal_to_number.values()) ### to increment the normal varlist
        for state_var, ts in faulty_vartrace:
            if state_var in normal_to_number:
                state_int = normal_to_number[state_var]
                faulty_trace_new += [[state_int, ts]]
            else:
                normal_to_number[state_var] = max_int + 1
                max_int += 1
                faulty_trace_new += [[max_int, ts]]

        #### check if the adapted trace is same as the original trace
        assert (len(faulty_trace_new) == len(faulty_trace))

        print('updated varlist with new events:',len(normal_to_number))
        # print(os.path.basename(path_to_trace).split('_')[1])
        # print(os.path.basename(path_to_trace).split('_')[1] in path_to_varlist)
        print('writing varlist to:', path_to_varlist)
        print('writing trace to:', path_to_trace)
        ### save the corrected trace and varlist
        with open(path_to_trace, 'w') as f:
            json.dump(faulty_trace_new, f)

        with open(path_to_varlist, 'w') as f:
            json.dump(normal_to_number, f)

        print('\n')


In [ ]:
len(faulty_trace_new)

In [ ]:
len(normal_from_number.keys())

In [ ]:
max(normal_to_number.values())